# Модель с использованием библиотеки `keras`

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import regularizers

model = Sequential()
model.add(Dense(3600, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape = (3,)))
model.add(Dense(1))

model.compile(optimizer=Adam(),loss='mse')

In [ ]:
# просмотр построенной модели
print(model.summary())

In [ ]:
from functionXYZ import f_xy
import numpy as np


def prepare_random_data(degrees, elems_in_degree):
    """ Функция, генерирующая одинаковое число случайных элементов для каждой переданной степени.
    Обоснование. Если брать просто рандомные числа в пределах от [-1000; 1000],
    то чисел в пределах [-1;1] почти не будет, чисел в пределах [-10;10] будет очень мало
    и так далее по аналогии, что приведёт к необучению младших разрядов.
    Эту проблему и решает данная функция, возвращающая данные для тренировки модели с одинаковым числом нужных степеней.
    """
    
    for iters, degree in enumerate(degrees):
        degree_multyplier = pow(10, degree)
        X_data_tmp = np.random.rand(elems_in_degree, 2)*degree_multyplier*2-degree_multyplier
        X_data_tmp = np.append(X_data_tmp, np.ones((elems_in_degree, 1)), axis=1)
        X_data_tmp = X_data_tmp.reshape((elems_in_degree, 3))
        X_data = np.append(X_data, X_data_tmp, 0) if iters > 0 else X_data_tmp
    np.random.shuffle(X_data)
    y_data = np.array([
        f_xy(x[0], x[1]) for x in X_data
    ]).reshape((elems_in_degree*len(degrees), 1))
    return X_data, y_data

# это номера разрядов 10, в рамках которых будет обучаться модель (не совсем так, но приблежено к истине)
# не больше 4 разряда, на большее модель не хватает
degrees = [-1, 0, 0.5]  # на самом деле, модель нормально обучается не больше чем только на трёх последовательных разрядах, например 1, 2, 3
elems_in_degree = 5000  # число элементов на разряд
X_k, y_k = prepare_random_data(degrees, elems_in_degree)

In [ ]:
# Тренировка модели, 20% идут на проверку
# Выполнить плитку раз 6-8, каждый раз с обновлёнными тренировочными данными (=выполнив плитку выше) (до loss равным 70-100)
hist = model.fit(
    X_k,
    y_k,
    validation_split=0.2,
    epochs=150,
    batch_size=256,
)

In [ ]:
# сохранение модели
model.save('asl_model')

In [ ]:
# так очищали память в курсе от nvidia, здесь это выдаёт странную ошибку
import IPython


app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Использование обученной модели

In [ ]:
from tensorflow import keras


# загрузка модели
model = keras.models.load_model('asl_model')

In [ ]:
from functionXYZ import f_xy
import numpy as np


def check_model(X):
    model_result = model.predict(X)
    function_result = [[f_xy(x[0], x[1])] for x in X]
    for i in range(len(X)):
        if function_result[i][0]:
            error = np.absolute(
                (np.absolute(model_result[i][0])-np.absolute(function_result[i][0]))
                / (np.absolute(model_result[i][0])+np.absolute(function_result[i][0]))
            )*100
        else:
            error = np.absolute(
                (np.absolute(model_result[i][0])-np.absolute(function_result[i][0]))
            )*100
        print(
            f'Got: {model_result[i][0]}, want: {function_result[i][0]},'
            f' error: {error}%'
        )


# проверка нескольких примеров
# каждый пример = [x, y, 1]
check_X = [
    [23, 54, 1],
    [3, 7, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
    [0.32, 0.67, 1],
    [1, -2, 1],
    [1.01, -2.02, 1],
]

# этот пример также призван показать, что модель обучается только в конкретном диапозоне разрядов 10.
check_model(check_X)

# Модель буз использования библиотек (legacy code)

Построенная модель без использования библиотек трудно настраивается на данные, не входящие в диапозон \[-1;1\].  
Призвана показать, как строится простейшая модель.

In [ ]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(y):
    return y * (1.0 - y)


class NeuralNetwork:
    def __init__(self, params_number, h_number):
        self.weights1 = np.random.rand(params_number, h_number)
        self.weights2 = np.random.rand(h_number, 1)

    def feedforward(self, input_data):
        self.layer1 = sigmoid(np.dot(input_data, self.weights1))
        return sigmoid(np.dot(self.layer1, self.weights2))

    def backprop(self, input_data, output_data, check_data):
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        d_weights2 = np.dot(
            self.layer1.T,
            (2*(check_data - output_data) * sigmoid_derivative(output_data))
        )
        d_weights1 = np.dot(
            input_data.T,
            (np.dot(
                2*(check_data - output_data) * sigmoid_derivative(output_data),
                self.weights2.T
            ) * sigmoid_derivative(self.layer1))
        )
        # update the weights with the derivative (slope) of the loss function
        self.weights1 += d_weights1
        self.weights2 += d_weights2

    def train(self, epochs, input_data, check_data):
        output_data = np.zeros(input_data.shape[0])
        for _ in range(epochs):
            output_data = self.feedforward(input_data)
            self.backprop(input_data, output_data, check_data)

In [ ]:
def func(x, y):
    return (x-1)**2+(y+2)**2

h_number = 6
params_number = 2
nn = NeuralNetwork(params_number, h_number)

In [ ]:
epochs = 10000
values_number = 30
max_x = 10
# X = np.array([[0, 0], [0, 1], [1, 0], [1, 1], [-max_x, max_x]])
X = np.random.rand(values_number, 2)*max_x
y = np.array([
    func(x[0], x[1]) for x in X
]).reshape((values_number, 1))

max_el = max(np.amax(np.absolute(X)), np.amax(np.absolute(y)))
X = np.dot(X, 1/max_el)
y = np.dot(y, 1/max_el)

In [ ]:
nn.train(epochs, X, y)

In [ ]:
z = nn.feedforward(X)
z = np.dot(z, max_el)
y = np.dot(y, max_el)
for i in range(z.shape[0]):
    error = np.absolute(
        (np.absolute(z[i][0])-np.absolute(y[i][0]))
        / (np.absolute(z[i][0])+np.absolute(y[i][0]))
    )*100
    print(
        f'Got: {z[i][0]}, want: {y[i][0]},'
        f' error: {error}%'
    )